# Load Data

In [1]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import umap
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline, make_pipeline
from lightgbm import LGBMClassifier
import joblib

sys.path.append('../utils')
sys.path.append('..')

from EDA_utils import*


c:\Users\maria\.conda\envs\modelling\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_folder_path = Path("../Datasets/Processed")
new_format_data_path = Path("../Datasets/Raw/Twibot-20-new-format")
old_format_data_path = Path("../Datasets/Processed/ETL")
embeddings_data_folder_path =  Path("../Datasets/Processed/BERT_output")

In [3]:
graph_embs_names = joblib.load(data_folder_path/"graph_emb_names.pkl")
numerical_features = joblib.load(data_folder_path/"numerical_features_names.pkl")
categorical_features = joblib.load(data_folder_path/"categorical_features_names.pkl")
tweet_emb_names = joblib.load(data_folder_path/"tweet_emb_names.pkl")
description_emb_names = joblib.load(data_folder_path/"description_emb_names.pkl")
name_emb_names = joblib.load(data_folder_path/"profile_name_emb_names.pkl")
screen_name_emb_names = joblib.load(data_folder_path/"screen_name_emb_names.pkl")

In [6]:
tsvd_name_embs = [f"tsvd_name_embs{i}" for i in range(20)]
tsvd_screen_name_embs = [f"tsvd_screen_name_embs{i}" for i in range(20)]
tsvd_tweet_embs = [f"tsvd_tweets_embs{i}" for i in range(20)]
tsvd_description_embs = [f"tsvd_description_embs{i}" for i in range(20)]
all_tsvd_embs = tsvd_name_embs + tsvd_screen_name_embs + tsvd_tweet_embs + tsvd_description_embs


ssvd_name_embs = [f"ssvd_name_embs{i}" for i in range(20)]
ssvd_screen_name_embs = [f"ssvd_screen_name_embs{i}" for i in range(20)]
ssvd_tweet_embs = [f"ssvd_tweets_embs{i}" for i in range(20)]
ssvd_description_embs = [f"ssvd_description_embs{i}" for i in range(20)]
all_ssvd_embs = tsvd_name_embs + ssvd_screen_name_embs + ssvd_tweet_embs + ssvd_description_embs

In [7]:
profile_features = numerical_features + categorical_features
graph_features = graph_embs_names

all_features_ssvd = profile_features + graph_features + all_ssvd_embs
all_features_tsvd = profile_features + graph_features + all_tsvd_embs

In [8]:
df = pd.read_parquet(data_folder_path/"preprocessed_profile_and_text_features.parquet")

In [10]:
fs_svd = pd.read_parquet(data_folder_path/"feature_selection_tsvd_logs.parquet")
fs_supervised_svd = pd.read_parquet(data_folder_path/"feature_selection_ssvd_logs.parquet")

# Modelling

In [ ]:
train_mask = lambda d: (d["split"] == "train") & (d["random_number"] >= 0.2)
test_mask = lambda d: d["split"] == "test"
support_mask = lambda d: d["split"] == "support"
val_mask = lambda d: (d["split"] == "train") & (d["random_number"] < 0.2)

target = "label"


In [ ]:
!pip install shap -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 7.3 MB/s eta 0:00:00


In [ ]:
from sklearn.base import clone
import numpy as np
import pandas as pd
import shap
from tqdm import tqdm

from toolz import curry
from sklearn.metrics import roc_auc_score

@curry
def fast_metric_with_ci_(data, *, n_samples=100, ci_level=0.95,
                     prediction='prediction', target='target', weight='weight', metric_fn = roc_auc_score):

    data = data.assign(weight__=lambda df: df[weight] if weight is not None else 1)
    summary = (
        data
        .assign(
            prediction=lambda df: (1000 * df[prediction]).round(),
        )
        .groupby(["weight__", 'prediction', target])
        .size().to_frame("sample_size")
        .reset_index()
    )

    estimate = (
        summary
        .assign(weight__=lambda df: df["weight__"] * df['sample_size'])
        .pipe(lambda df: metric_fn(df[target], df['prediction'], sample_weight=df['weight__']))
    )

    bs_values = [
        summary
        .assign(weight__=lambda df: df["weight__"] * np.random.poisson(df['sample_size']))
        .pipe(lambda df: metric_fn(df[target], df['prediction'], sample_weight=df['weight__']))
    for _ in range(n_samples)]

    lo, hi = bootstrap_ci(estimate, bs_values, ci_level=ci_level)

    return pd.Series(dict(
        estimate=estimate,
        ci_upper=hi,
        ci_lower=lo,
        model=prediction
    ))


def bootstrap_ci(sample_estimate, bootstrap_estimates, ci_level=0.95):
    lo = 2 * sample_estimate - np.quantile(bootstrap_estimates, (1 + ci_level) / 2)
    hi = 2 * sample_estimate - np.quantile(bootstrap_estimates, (1 - ci_level) / 2)
    return lo, hi



@curry
def fast_delta_metric_with_ci_(data, baseline, challenger, *, n_samples=100, ci_level=0.95,
                           target='target', weight='weight', metric_fn = roc_auc_score):

    data = data.assign(weight__=lambda df: df[weight] if weight is not None else 1)

    summary = (
        data
        .assign(**{
            baseline: lambda df: (1000 * df[baseline]).round(),
            challenger: lambda df: (1000 * df[challenger]).round(),
        })
        .groupby(["weight__", baseline, challenger, target])
        .size().to_frame("sample_size")
        .reset_index()
    )


    def delta_auc(df):
        challenger_auc = metric_fn(df[target], df[challenger], sample_weight=df['weight__'])
        baseline_auc = metric_fn(df[target], df[baseline], sample_weight=df['weight__'])
        return challenger_auc - baseline_auc

    estimate = (
        summary
        .assign(weight__=lambda df: df["weight__"] * df['sample_size'])
        .pipe(delta_auc)
    )

    bs_values = [
        summary
        .assign(weight__=lambda df: df["weight__"] * np.random.poisson(df['sample_size']))
        .pipe(delta_auc)
    for _ in range(n_samples)]

    lo, hi = bootstrap_ci(estimate, bs_values, ci_level=ci_level)

    return pd.Series(dict(
        estimate=estimate,
        ci_upper=hi,
        ci_lower=lo,
        model=challenger
    ))


@curry
def fast_delta_metric_with_ci(data, baseline, challengers, target, *, n_samples=100, ci_level=0.95, weight='weight', metric_fn = roc_auc_score):

    fn = fast_delta_metric_with_ci_(
        baseline=baseline,
        n_samples=n_samples,
        ci_level=ci_level,
        target=target,
        weight=weight,
        metric_fn=metric_fn
      )

    all_values = [fn(data=data,challenger=c) for c in challengers]

    return pd.DataFrame(all_values)

@curry
def fast_metric_with_ci(data, predictions, target, *, n_samples=100, ci_level=0.95, weight='weight', metric_fn = roc_auc_score):

    fn = fast_metric_with_ci_(
        target=target,
        n_samples=n_samples,
        ci_level=ci_level,
        weight=weight,
        metric_fn=metric_fn
      )

    all_values = [fn(data=data,prediction=p) for p in predictions]

    return pd.DataFrame(all_values)

def log_odds_to_proba(x):
  return 1/(1+np.exp(-x))

def proba_to_log_odds(p):
  return np.log(p/(1-p))


def backwards_shap_feature_selection(
    model,
    df_train,
    df_val,
    candidate_features_for_removal,
    target,
    null_hypothesis = "feature_is_good",
    fixed_features=[],
    n_features_sample=None,
    extra_validation_sets = {},
    sample_weight=None,
    metric_fn = roc_auc_score,
    bootstrap_samples=20,
    ci_level=0.8,
    max_iter = 10,
    patience=0,
    max_removals_per_run=None
):

  """
  """
  #TODO: implement two null hypothesis strategies. currently only "all_features_are_good"


  #check key names
  valid_nulls = ["feature_is_good","feature_is_bad"]
  if not null_hypothesis in valid_nulls:
      raise(ValueError(f"null_hypothesis should be one of {valid_nulls}, got {null_hypothesis}"))

  keys_intersections = set(extra_validation_sets.keys()) & set(candidate_features_for_removal + fixed_features)
  if keys_intersections:
    raise ValueError(f"extra_validation_sets names should not match names of features. Found {keys_intersections}")

  keys_intersections = keys_intersections & set(["metric", "error-contribution"])
  if keys_intersections:
    raise ValueError(f"extra_validation_sets names or feature names should not be 'metric' or 'error-contribution'. Found {keys_intersections}")

  all_logs = []
  p=0
  for i in tqdm(range(max_iter)):

    #set all features
    all_features = candidate_features_for_removal + fixed_features

    if len(all_features) == 0:
      break

    if (n_features_sample is None) or (len(all_features) <= n_features_sample):
      features_to_use = all_features
    else:
      features_to_use = np.random.choice(all_features, n_features_sample, replace=False)

    run_logs = _backwards_shap_feature_selection(
        model=clone(model),
        df_train=df_train,
        df_val=df_val,
        all_features=features_to_use,
        extra_validation_sets=extra_validation_sets,
        target=target,
        sample_weight=sample_weight,
        metric_fn=metric_fn,
        bootstrap_samples=bootstrap_samples,
        ci_level=ci_level,
    )

    if null_hypothesis == "feature_is_good":
      features_to_remove = (
          run_logs
          [lambda d: d["ci_lower"] > 0]
          [lambda d: d["metric"] == "error-contribution"]
          [lambda d: ~d["model"].isin(fixed_features)]
          .sort_values(by = "ci_lower", ascending=False)
      )
    else:
      features_to_remove = (
          run_logs
          [lambda d: d["ci_upper"] > 0]
          [lambda d: d["metric"] == "error-contribution"]
          [lambda d: ~d["model"].isin(fixed_features)]
          .sort_values(by = "ci_upper", ascending=False)
      )


    if max_removals_per_run is not None:
      features_to_remove = features_to_remove.iloc[:max_removals_per_run]

    features_to_remove = features_to_remove["model"].values.tolist() #model means the model without the feature

    run_logs["run_index"] = i
    run_logs["n_features"] = (run_logs["metric"] == "error-contribution").sum()
    run_logs["removed_features"] = str(features_to_remove)
    run_logs["n_features_removed"] = len(features_to_remove)
    all_logs.append(run_logs)

    if len(features_to_remove) == 0:
      if patience:
        if p >= patience:
          break
        else:
          p+=1
      else:
        break
    #update features for the next iteration
    candidate_features_for_removal = [i for i in candidate_features_for_removal if not i in features_to_remove]

    #update counters
    p=0
    i+=1

  #calculate fs logs for full set of features in case of sub sampling
  if (n_features_sample is not None) and (len(all_features) > n_features_sample):
      run_logs = _backwards_shap_feature_selection(
          model=clone(model),
          df_train=df_train,
          df_val=df_val,
          all_features=all_features,
          extra_validation_sets=extra_validation_sets,
          target=target,
          sample_weight=sample_weight,
          metric_fn=metric_fn,
          bootstrap_samples=bootstrap_samples,
          ci_level=ci_level,
      )
      run_logs["run_index"] = i + 1
      run_logs["n_features"] = (run_logs["metric"] == "error-contribution").sum()
      run_logs["removed_features"] = str([])
      run_logs["n_features_removed"] = 0
      all_logs.append(run_logs)

  return pd.concat(all_logs, ignore_index=True)


def _backwards_shap_feature_selection(
    model,
    df_train,
    df_val,
    all_features,
    extra_validation_sets,
    target,
    sample_weight,
    metric_fn,
    bootstrap_samples,
    ci_level,
):

  #train model
  model.fit(
      df_train[all_features],
      df_train[target],
      sample_weight=sample_weight
  )


  #calculate shap
  explainer = shap.TreeExplainer(model)
  shap_values_val = explainer.shap_values(df_val[all_features])[-1]

  #make raw preds
  raw_preds_val = proba_to_log_odds(model.predict_proba(df_val[all_features])[:,-1])

  #score without feature
  scores_df = pd.DataFrame(
      log_odds_to_proba(raw_preds_val.reshape(-1,1) - shap_values_val),
      columns = all_features
  )

  #add extra columns
  scores_df["val_set"] = raw_preds_val
  scores_df[target] = df_val[target].values
  if sample_weight is not None:
    df_val[sample_weight].values


  #deltas
  error_contributions_with_ci = fast_delta_metric_with_ci(
      scores_df,
      baseline="val_set",
      challengers=all_features,
      n_samples=bootstrap_samples,
      ci_level=ci_level,
      target=target,
      weight=sample_weight,
      metric_fn = metric_fn
    ).assign(metric="error-contribution")

  #current setup auc
  metric = fast_metric_with_ci(
      scores_df,
      predictions=["val_set"],
      n_samples=bootstrap_samples,
      ci_level=ci_level,
      target=target,
      weight=sample_weight,
      metric_fn = metric_fn
    ).assign(metric="metric", used_features=str(all_features))

  extra_val_logs = []
  for k,d in extra_validation_sets.items():
    extra_val_logs.append(
        fast_metric_with_ci(
          d.assign(**{k:lambda d: model.predict_proba(d[all_features])[:,-1], "weight__":lambda d: d[sample_weight] if sample_weight is not None else 1}),
          predictions=[k],
          n_samples=bootstrap_samples,
          ci_level=ci_level,
          target=target,
          weight="weight__",
          metric_fn = metric_fn
      ).assign(metric="metric", used_features=str(all_features))
    )



  return pd.concat([error_contributions_with_ci, metric, *extra_val_logs], ignore_index = True)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

ssvd_fs_logs = backwards_shap_feature_selection(
    LGBMClassifier(n_jobs = -1, min_samples_leaf = 5, verbose = 0),
    df[train_mask],
    df[val_mask],
    candidate_features_for_removal = all_features_ssvd,
    target=target,
    null_hypothesis="feature_is_good",
    n_features_sample=40,
    extra_validation_sets={"test_set": df[test_mask]},
    fixed_features=[],
    sample_weight=None,
    metric_fn = roc_auc_score,
    bootstrap_samples=30,
    ci_level=0.80,
    max_iter=30,
    patience=2,
    max_removals_per_run=None
)

  0%|          | 0/50 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=5. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=5. Current value: min_data_in_leaf=5


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=5. Current value: min_data_in_leaf=5


  2%|▏         | 1/50 [00:58<48:02, 58.83s/it]

...

 44%|████▍     | 22/50 [17:22<19:42, 42.22s/it]

[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=5. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=5. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=5. Current value: min_data_in_leaf=5


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=5. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=5. Current value: min_data_in_leaf=5


 44%|████▍     | 22/50 [18:02<22:57, 49.21s/it]


In [ ]:
ssvd_fs_logs[lambda d: d.model == "test_set"]

,estimate,ci_upper,ci_lower,model,metric,used_features,run_index,n_features,removed_features,n_features_removed
51,0.872908,0.884868,0.853435,test_set,metric,['ssvd_tweets_embs4' 'ssvd_tweets_embs1' 'favo...,0,50,"['follow_embs2', 'ssvd_screen_name_embs19']",2
103,0.916616,0.931107,0.907021,test_set,metric,['followed_embs4' 'follow_embs5' 'friend_embs8...,1,50,"['followed_embs7', 'ssvd_screen_name_embs3']",2
155,0.910644,0.928563,0.888391,test_set,metric,['tsvd_name_embs12' 'tsvd_name_embs13' 'ssvd_s...,2,50,['friend_embs8'],1
207,0.908762,0.916747,0.896381,test_set,metric,['ssvd_description_embs19' 'ssvd_tweets_embs8'...,3,50,['ssvd_screen_name_embs11'],1
259,0.918271,0.936614,0.900632,test_set,metric,['favourites_count' 'tsvd_name_embs9' 'profile...,4,50,[],0
311,0.762127,0.788592,0.735984,test_set,metric,['ssvd_description_embs10' 'ssvd_screen_name_e...,5,50,"['followed_embs1', 'friend_embs2', 'friend_emb...",11
363,0.853269,0.883618,0.836309,test_set,metric,['default_profile' 'ssvd_screen_name_embs17' '...,6,50,"['tsvd_name_embs10', 'friend_embs3']",2
415,0.876529,0.892418,0.859736,test_set,metric,['ssvd_screen_name_embs12' 'tsvd_name_embs16' ...,7,50,['followed_embs6'],1
467,0.864218,0.885468,0.843110,test_set,metric,['ssvd_tweets_embs17' 'ssvd_tweets_embs11' 'ss...,8,50,"['tsvd_name_embs5', 'ssvd_tweets_embs10', 'fol...",3
519,0.923661,0.936350,0.913410,test_set,metric,['ssvd_description_embs15' 'ssvd_tweets_embs6'...,9,50,"['ssvd_description_embs18', 'ssvd_screen_name_...",2
